In [2]:
!pip install vaderSentiment


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.7 MB/s eta 0:00:0000:01


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import cohen_kappa_score
from sklearn.utils.class_weight import compute_class_weight
from PIL import Image, ImageStat
import os
import re
import emoji
from imblearn.over_sampling import SMOTE
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

# Шляхи до файлів
print("Завантаження шляхів до файлів...")
TRAIN_CSV = '/kaggle/input/deep-learning-for-computer-vision-and-nlp-2024-10/train.csv'
TEST_CSV = '/kaggle/input/deep-learning-for-computer-vision-and-nlp-2024-10/test.csv'
TRAIN_IMAGES_PATH = '/kaggle/input/deep-learning-for-computer-vision-and-nlp-2024-10/images/images/train'
TEST_IMAGES_PATH = '/kaggle/input/deep-learning-for-computer-vision-and-nlp-2024-10/images/images/test'

# Завантаження даних
print("Завантаження даних...")
train_data = pd.read_csv(TRAIN_CSV)
test_data = pd.read_csv(TEST_CSV)
print("Розмір тренувального набору:", train_data.shape)
print("Розмір тестового набору:", test_data.shape)

# Заповнення пропущених значень у колонці 'Description' і конвертація в рядки
train_data['Description'] = train_data['Description'].fillna('').astype(str)
test_data['Description'] = test_data['Description'].fillna('').astype(str)


# Функція для обчислення середньої яскравості зображення
def calculate_brightness(image_path):
    try:
        img = Image.open(image_path).convert('L')  # Конвертація в чорно-біле зображення
        stat = ImageStat.Stat(img)
        return stat.mean[0]  # Середня яскравість
    except:
        return 0


# Функція для обробки зображення через ResNet50
def process_image(image_path):
    try:
        img = image.load_img(image_path, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)
        return img_data
    except:
        return np.zeros((224, 224, 3))


# Завантаження ResNet50 для отримання ознак
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
model = Model(inputs=base_model.input, outputs=x)


# Функція для витягнення ознак зображень через ResNet50
def extract_image_features_resnet(pet_id, path):
    img_folder = f'{path}/{pet_id}'
    if not os.path.exists(img_folder):
        return np.zeros((2048,))  # Якщо немає зображень

    image_files = os.listdir(img_folder)
    if len(image_files) == 0:
        return np.zeros((2048,))

    # Обробляємо перше зображення
    img_path = f'{img_folder}/{image_files[0]}'
    img_data = process_image(img_path)
    features = model.predict(img_data)

    return features.flatten()


# Створюємо метаінформацію для тренувальних даних
print("Обробка зображень для метаінформації через ResNet50...")
train_data['image_features'] = train_data['PetID'].apply(lambda x: extract_image_features_resnet(x, TRAIN_IMAGES_PATH))

# Створюємо метаінформацію для тестових даних
print("Обробка тестових зображень для метаінформації через ResNet50...")
test_data['image_features'] = test_data['PetID'].apply(lambda x: extract_image_features_resnet(x, TEST_IMAGES_PATH))


# Функція для виділення віку
def extract_age(description):
    age_search = re.search(r'\b\d{1,2}\s?(months?|years?)\b', description.lower())
    if age_search:
        age = age_search.group(0)
        # Конвертація віку в місяці або роки
        if 'year' in age:
            return int(re.search(r'\d+', age).group(0)) * 12  # Конвертуємо в місяці
        elif 'month' in age:
            return int(re.search(r'\d+', age).group(0))  # Залишаємо в місяцях
    return np.nan


# Функція для виділення породи (тільки наявність або відсутність згадки про породу)
def extract_breed_presence(description):
    breed_search = re.search(r'\b(mixed breed|poodle|labrador|bulldog|cat|dog)\b', description.lower())
    return 1 if breed_search else 0


# Функція для виділення стану здоров'я
def extract_health_status(description):
    health_search = re.search(r'\b(healthy|vaccinated|neutered|sick)\b', description.lower())
    if health_search:
        return health_search.group(0)
    return 'unknown'


# Функція для підрахунку кількості емодзі в тексті
def count_emojis(text):
    return len([char for char in text if char in emoji.EMOJI_DATA])


# Аналіз емоційного забарвлення тексту за допомогою VADER
analyzer = SentimentIntensityAnalyzer()


def extract_sentiment(description):
    sentiment = analyzer.polarity_scores(description)
    return pd.Series([sentiment['pos'], sentiment['neu'], sentiment['neg'], sentiment['compound']])


# Додаємо нові ознаки до датасету
train_data['Age'] = train_data['Description'].apply(extract_age)
train_data['Breed_Presence'] = train_data['Description'].apply(extract_breed_presence)
train_data['Health_Status'] = train_data['Description'].apply(extract_health_status)
train_data['emoji_count'] = train_data['Description'].apply(count_emojis)

test_data['Age'] = test_data['Description'].apply(extract_age)
test_data['Breed_Presence'] = test_data['Description'].apply(extract_breed_presence)
test_data['Health_Status'] = test_data['Description'].apply(extract_health_status)
test_data['emoji_count'] = test_data['Description'].apply(count_emojis)

# Додаємо ознаки емоційного забарвлення (позитивний, нейтральний, негативний, комбінований)
train_data[['sentiment_pos', 'sentiment_neu', 'sentiment_neg', 'sentiment_compound']] = train_data['Description'].apply(
    extract_sentiment)
test_data[['sentiment_pos', 'sentiment_neu', 'sentiment_neg', 'sentiment_compound']] = test_data['Description'].apply(
    extract_sentiment)

# Додаємо бінарну ознаку для відсутності віку
train_data['Age_missing'] = train_data['Age'].isna().astype(int)
test_data['Age_missing'] = test_data['Age'].isna().astype(int)

# Замінюємо пропущені значення в Age спеціальним кодом (-1 для позначення невідомого віку)
train_data['Age'] = train_data['Age'].fillna(-1)
test_data['Age'] = test_data['Age'].fillna(-1)

# Обробка тексту за допомогою TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
train_text_data = vectorizer.fit_transform(train_data['Description'].fillna(''))
test_text_data = vectorizer.transform(test_data['Description'].fillna(''))

# Підготовка повного набору ознак
train_features = np.hstack([
    train_text_data.toarray(),
    np.vstack(train_data['image_features'].values),
    train_data[['Breed_Presence', 'Age', 'emoji_count', 'Age_missing',
                'sentiment_pos', 'sentiment_neu', 'sentiment_neg', 'sentiment_compound']].values
])

test_features = np.hstack([
    test_text_data.toarray(),
    np.vstack(test_data['image_features'].values),
    test_data[['Breed_Presence', 'Age', 'emoji_count', 'Age_missing',
               'sentiment_pos', 'sentiment_neu', 'sentiment_neg', 'sentiment_compound']].values
])

# Стандартизація даних
scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)
test_features_scaled = scaler.transform(test_features)

# Розділення на тренувальні та валідаційні набори
X_train, X_val, y_train, y_val = train_test_split(train_features_scaled, train_data['AdoptionSpeed'], test_size=0.2,
                                                  random_state=42)

# Балансування класів за допомогою SMOTE (якщо потрібно)
print("Балансування класів за допомогою SMOTE...")
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Обчислення ваг класів
print("Обчислення ваг класів...")
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

# Тренування моделі LightGBM
lgb_model = lgb.LGBMRegressor(objective='regression', metric='rmse', num_leaves=31, learning_rate=0.05,
                              n_estimators=5000)

print("Тренування моделі LightGBM...")
lgb_model.fit(X_train_balanced, y_train_balanced)

# Прогнозування на валідаційних даних
y_val_pred = lgb_model.predict(X_val)
y_val_pred_rounded = np.round(y_val_pred).astype(int)
y_val_pred_rounded = np.clip(y_val_pred_rounded, 1, 4)

# Оцінка метрики Kappa на валідаційних даних
kappa = cohen_kappa_score(y_val, y_val_pred_rounded, weights='quadratic')
print("Quadratic Weighted Kappa на валідаційних даних:", kappa)

# Прогнозування на тестових даних
print("Прогнозування на тестових даних...")
predictions = lgb_model.predict(test_features_scaled)
predictions_rounded = np.round(predictions).astype(int)
predictions_rounded = np.clip(predictions_rounded, 1, 4)

# Формування файлу submission.csv
submission = pd.DataFrame({'PetID': test_data['PetID'], 'AdoptionSpeed': predictions_rounded})
submission.to_csv('submission.csv', index=False)
print("Файл submission.csv сформовано.")


Завантаження шляхів до файлів...
Завантаження даних...
Розмір тренувального набору: (6431, 3)
Розмір тестового набору: (1891, 2)
Обробка зображень для метаінформації через ResNet50...
Обробка тестових зображень для метаінформації через ResNet50...
Балансування класів за допомогою SMOTE...
Обчислення ваг класів...
Тренування моделі LightGBM...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.104040 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 79343
[LightGBM] [Info] Number of data points in the train set: 6804, number of used features: 1709
[LightGBM] [Info] Start training from score 2.500000
Quadratic Weighted Kappa на валідаційних даних: 0.3196684195591404
Прогнозування на тестових даних...
Файл submission.csv сформовано.
